# auto feature engineering on NYC Taxi Fare

In [1]:
import sys
sys.path.append('../../../')

import pandas as pd
from utils import Timer

train_data = pd.read_csv("nyc_taxi_fare_cleaned.csv", nrows=10000)

from pyrecdp.autofe import AutoFE
fwg = AutoFE(dataset=train_data, label="fare_amount")

2022-12-20 22:43:51,543 featuretools - WARNING    Featuretools failed to load "nlp_primitives" primitives from "nlp_primitives". For a full stack trace, set logging to debug.
2022-12-20 22:43:51,552 featuretools - WARNING    Featuretools failed to load plugin nlp_primitives from library nlp_primitives. For a full stack trace, set logging to debug.


/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyrecdp/primitives/generators/statics.py:27: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyrecdp/primitives/generators/statics.py:27: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



# auto feature engineering on twitter recsys

In [3]:
import sys
sys.path.append('../../../')

import pandas as pd
from utils import Timer

train_data = pd.read_csv("twitter_recsys_cleaned.csv", nrows=10000)

from pyrecdp.autofe import AutoFE
fwg = AutoFE(dataset=train_data, label="reply")

/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyrecdp/primitives/generators/statics.py:26: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyrecdp/primitives/generators/statics.py:26: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyrecdp/primitives/generators/statics.py:26: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` t

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_311655/3956099255.py", line 10, in <module>
    fwg = AutoFE(dataset=train_data, label="reply")
  File "/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyrecdp/autofe/AutoFE.py", line 57, in __init__
    self.transformed_df_profile = FeatureProfiler(self.transformed_feature, self.data[self.label]).visualize_analyze()
  File "/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyrecdp/autofe/FeatureProfiler.py", line 72, in visualize_analyze
    return FeatureVisulizer(self.fit_analyze())
  File "/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyrecdp/autofe/FeatureProfiler.py", line 64, in fit_analyze
    feature_data = TypeInferFeatureGenerator().fit_transform(feature_data)
  File "/home/vmagent/app/recdp/examples/notebooks/autofe/../../../pyr

In [ ]:
from pyrecdp.widgets import BaseWidget, TabWidget

new_view = BaseWidget(display_flag=False)
fwg.main_view.append('new_view', new_view)


In [ ]:
print(fwg.get_transform_pipeline())

In [ ]:
fwg.get_feature_list()

# Backup

In [ ]:
import pandas as pd
from utils import Timer
import os

def clean_df(df, label = 'reply'):
    df['tweet_timestamp'] = pd.to_datetime(df['tweet_timestamp'],unit='s')
    df['engaging_user_account_creation'] = pd.to_datetime(df['engaging_user_account_creation'],unit='s')
    df['engaged_with_user_account_creation'] = pd.to_datetime(df['engaged_with_user_account_creation'],unit='s')
    
    df['reply'] = df['reply_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['retweet'] = df['retweet_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['comment'] = df['retweet_with_comment_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df['like'] = df['like_timestamp'].apply(lambda x: 1 if x>0 else 0).astype('int32')
    df = df.drop(columns=['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp'])
    if 'tokens' in df.columns:
        df = df.drop(columns=['tokens'])
    return df

def apply_bert_decode(df):
    from transformers import BertTokenizer
    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
    df["tweet"] = df["text_tokens"].progress_apply(lambda x: tokenizer.decode([int(n) for n in x.split('\t')]))
    df = df.drop(columns=['text_tokens'])
    return df

with Timer(f"read data"):
    schema = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains", "tweet_type",
        "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count", "engaged_with_user_following_count",
        "engaged_with_user_is_verified", "engaged_with_user_account_creation", "engaging_user_id", "engaging_user_follower_count",
        "enaging_user_following_count", "enaging_user_is_verified", "engaging_user_account_creation", "engagee_follows_engager",
        "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
    train_data = pd.read_csv("twitter_recsys.csv", sep='\x01', names = schema, header=None,)
with Timer(f"clean data"):
    train_data = clean_df(train_data)
with Timer("write_data"):
    train_data.to_csv("twitter_recsys_cleaned.csv", index = False)